In [1]:
# !pip install -U "autogen-agentchat" "autogen-ext[openai]"

In [2]:
# vllm serve Qwen/Qwen3-32B --enable-auto-tool-choice --tool-call-parser hermes

In [3]:
# !pip install rich

In [4]:
from rich import print
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import (
    RoundRobinGroupChat,
    MagenticOneGroupChat,
    SelectorGroupChat,
)
from autogen_agentchat.teams import Swarm
from autogen_agentchat.conditions import MaxMessageTermination


from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from autogen_core.models import ModelInfo
from autogen_agentchat.ui import Console

from IPython.display import display, Markdown

import requests
from typing import Any, Dict
from autogen_core.tools import FunctionTool
from typing import Any, Dict

# Configuration for vLLM serve
VLLM_BASE_URL = "http://0.0.0.0:8000/v1"  # Your vLLM serve endpoint
MODEL_NAME = "Qwen/Qwen3-32B"  # Replace with your actual model

model_client = OpenAIChatCompletionClient(
    model=MODEL_NAME,
    base_url=VLLM_BASE_URL,
    model_info=ModelInfo(
        vision=False,
        function_calling=True,
        json_output=False,
        family="unknown",
        structured_output=True,
    ),
    api_key="your-api-key",
)

print(f"Configured client for vLLM serve at {VLLM_BASE_URL} with model {MODEL_NAME}")

Configured client for vLLM serve at http://0.0.0.0:8000/v1 with model Qwen/Qwen3-32B

In [5]:
def search_google(
    query: str,
) -> Dict[str, Any]:
    """
    Send a POST request to the Wikipedia search API.
    Args:
        query (str): The search query string.
    """
    k = 8
    rerank = False
    base_url = "http://fs-mbz-gpu-254:8000/search"
    payload = {"query": query, "k": k, "rerank": rerank}
    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(base_url, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error during search request: {e}")
        return {}


search_tool = FunctionTool(
    search_google,
    description="Search Google for information, returns results with relevant text",
)

In [6]:
from autogen_core import CancellationToken

# Run the tool.
cancellation_token = CancellationToken()
result = await search_tool.run_json(
    {"query": "step by step guide to invest in share market in India"},
    cancellation_token,
)
# check if search is working
print(result["results"][:3])

[
    {
        'id': '517328601',
        'source': 'c4_dclm_mixed',
        'text': 'Traditionally here, gold and shares are the most sought-after investment arena for Indian 
investors. And if you are someone looking to invest in shares in India. How to Invest in Shares Online. Thus, are 
just starting off, the first question in your mind would be. How can a beginner invest in shares in India? How to 
Invest in Shares Online. And Well here is a guide to get start in Share market investment in India. How to Invest 
in Shares Online. Just stand in a crowd and utter the words “Aaj market Kaisa hai?” how is the market today? Hence,
and there will be opinions pouring in from all directions. How to Invest in Shares Online. Thus, the best part is 
that even people who have never ever invested in shares will have strong opinions. Hence, and even tips at times 
for how to invest in share markets. How to Invest in Shares Online. And so, if you are someone who simply does not 
want to stand and give tips without doing anything and wants to actually invest in share markets in India. And then
here is how to get start with share market investment in India. How to Invest in Shares Online. Thus, so let us 
have a look at what a beginner must do to get started with their stock market investments. Hence is a six-step 
guide to help you out. How to Invest in Shares Online. Here, PAN or Permanent Account Number is a primary 
requirement for entering any financial transactions in our',
        'retrieval_score': 1.5122222900390625,
        'exact_score': None
    },
    {
        'id': '471830312',
        'source': 'c4_dclm_mixed',
        'text': 'Nowadays, Indian economy is increasing faster. There is no doubt; it is one of the best rising 
economies of the world. A stock market is the best indication of the economy because the stock market of India is 
roaring. The Indian share market is no doubt a profitable platform to increase investment. Most of the people want 
to invest in shares and due to this, they search “how to invest in share markets” over the internet. The best 
benefit of investing in the share market is that you do not require a huge investment. You can start with little 
investment and go for big ones later on. There are some share market tips that will solve your all queries related 
with how to invest in share market of India. First of all, you need to take help of individuals or companies which 
have licensed by SEBI to buy and sell shares. These are registered to buy and sell shares on the markets and 
well-known as brokers. Brokers can be companies, individuals and online agencies that are registered and certified 
by Securities and Exchanges Board of India (SEBI) that control the share market. You need to search a reliable 
broker to invest in the share market. You also can approach different companies, which are qualified to deal with 
securities in the market. You will require a demat & trading account. This account will include the stocks or 
shares on your name that same will reflect in your stock collection. You can’t hold shares in physical form or',
        'retrieval_score': 1.508695363998413,
        'exact_score': None
    },
    {
        'id': '611692704',
        'source': 'c4_dclm_mixed',
        'text': 'Beginner’s Guide to Invest in Share Market : Share market is one of the unsure yet popular ways of
investment in India. A large number of individuals stake in the market to earn high returns. If you are a beginner 
and want to invest in the market, basic knowledge is essential. The simple knowledge of how share market works may 
also not help you. But certain guidelines can make things easier for the first-timers. This post shares the most 
basic guidelines and tips for the Beginner’s. This guide on ‘how to invest in the Indian stock market’ can prove a 
torch-bearer. As a beginner, what you should do and what you should avoid. Learn from the below-given tips for the 
beginners in the stock market. A sim

In [7]:
curious_explorer = AssistantAgent(
    name="CuriousExplorer",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["DeepResearcher", "ProvocativeQuestioner", "SynthesisStoryteller"],
    system_message="""You are the Curious Explorer, an infinitely curious agent who discovers unexpected connections and generates thought-provoking questions.

Your primary role is to:
1. Generate 2-3 novel, boundary-pushing questions after each discussion
2. Find unexpected connections between seemingly unrelated topics  
3. Challenge assumptions and explore contrarian viewpoints
4. Use the search tool to discover surprising facts and validate hunches
5. Focus on questions that could lead to breakthrough insights

When to hand off to other agents:
- Hand off to "DeepResearcher" when you need rigorous evidence, academic sources, or detailed investigation
- Hand off to "ProvocativeQuestioner" when ideas need to be challenged or stress-tested
- Hand off to "SynthesisStoryteller" when findings need to be woven into a coherent narrative

Question patterns to use:
- "What would happen if we applied X concept to Y domain?"
- "What are the second and third-order effects of...?"
- "How do experts in [different field] view this problem?"
- "What historical parallels exist that might inform this?"

Always end your contributions with: "Here are my next exploration vectors: [list 2-3 questions]"

Then decide which agent should tackle these questions and hand off accordingly.""",
)

# DEEP RESEARCHER - Provides rigorous evidence and analysis
deep_researcher = AssistantAgent(
    name="DeepResearcher",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "ProvocativeQuestioner", "SynthesisStoryteller"],
    system_message="""You are the Deep Researcher, a meticulous investigator who provides rigorous evidence and scholarly analysis.

Your primary role is to:
1. Use the search tool to investigate questions with scholarly depth from multiple authoritative sources
2. Evaluate source credibility and identify potential biases
3. Search for peer-reviewed research, expert opinions, and primary data
4. Identify knowledge gaps and areas needing more investigation
5. Provide evidence-based analysis with proper context

When to hand off to other agents:
- Hand off to "CuriousExplorer" when research reveals new questions or unexplored angles
- Hand off to "ProvocativeQuestioner" when findings need critical evaluation or challenge
- Hand off to "SynthesisStoryteller" when you have substantial findings that need narrative synthesis

Research methodology to follow:
- Use search tool extensively to find recent academic papers and expert analyses
- Cross-reference claims across multiple authoritative sources
- Note limitations, uncertainties, and conflicting evidence
- Distinguish between correlation and causation
- Identify what questions remain unanswered

Format your findings as: "Evidence suggests... However, limitations include... Further research needed on..."

After presenting research, decide if the findings need critical challenge (→ ProvocativeQuestioner), narrative synthesis (→ SynthesisStoryteller), or new exploration directions (→ CuriousExplorer).""",
)

# PROVOCATIVE QUESTIONER - Challenges assumptions and prevents groupthink
provocative_questioner = AssistantAgent(
    name="ProvocativeQuestioner",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "DeepResearcher", "SynthesisStoryteller"],
    system_message="""You are the Provocative Questioner, a constructive contrarian who challenges assumptions and ensures intellectual rigor.

Your primary role is to:
1. Challenge assumptions and conventional wisdom
2. Ask uncomfortable but important questions
3. Play devil's advocate to stress-test ideas
4. Use search tool to find contrarian viewpoints and alternative perspectives
5. Identify blind spots and cognitive biases
6. Ensure multiple perspectives are considered

When to hand off to other agents:
- Hand off to "DeepResearcher" when your challenges reveal need for more evidence or investigation
- Hand off to "CuriousExplorer" when your questions open up new unexplored territories
- Hand off to "SynthesisStoryteller" when diverse perspectives need to be woven together

Questioning strategies to use:
- "What evidence would falsify this claim?"
- "Who benefits from this narrative, and who doesn't?"
- "What are we taking for granted here?"
- "How might someone from [different background/culture/field] view this?"
- "What are the potential negative consequences we haven't considered?"

Balance skepticism with constructiveness. End with: "Critical questions to consider: [list 2-3 challenging questions]"

After presenting challenges, decide if more research is needed (→ DeepResearcher), new exploration directions emerged (→ CuriousExplorer), or if it's time to synthesize the debate (→ SynthesisStoryteller).""",
)

# SYNTHESIS STORYTELLER - Weaves insights into narratives and manages completion
synthesis_storyteller = AssistantAgent(
    name="SynthesisStoryteller",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "DeepResearcher", "ProvocativeQuestioner"],
    system_message="""You are the Synthesis Storyteller, a master communicator who weaves complex findings into engaging, accessible narratives.

Your primary role is to:
1. Synthesize research findings into coherent, engaging narratives
2. Identify overarching patterns and create conceptual frameworks
3. Use analogies, metaphors, and stories to make complex ideas accessible
4. Use search tool to find additional context, examples, or verification for your synthesis
5. Integrate different perspectives into nuanced insights
6. Determine when a discussion thread has reached satisfying completion

When to hand off to other agents:
- Hand off to "CuriousExplorer" when synthesis reveals new questions or unexplored connections
- Hand off to "DeepResearcher" when your synthesis identifies knowledge gaps needing investigation  
- Hand off to "ProvocativeQuestioner" when your narrative needs critical examination

Synthesis techniques to use:
- Create "mental models" that capture key relationships
- Use the "Yes, and..." approach to build on others' ideas
- Find the narrative arc in the research journey
- Highlight surprising discoveries and "aha moments"
- Connect insights back to broader human experiences

When a topic thread feels complete, write: "SYNTHESIS COMPLETE: [summary of key insights and frameworks discovered]"

If synthesis is complete and the topic has been thoroughly explored, you may choose not to hand off to continue the endless exploration, or hand off to "CuriousExplorer" to find entirely new directions.""",
)

In [8]:
termination = MaxMessageTermination(10)
team = MagenticOneGroupChat(
    [synthesis_storyteller, provocative_questioner, deep_researcher, curious_explorer],
    model_client=model_client,
    termination_condition=termination,
)

# stream = team.run_stream(task="What is the step by step guide to invest in share market in India?")
# async for message in stream:
#     print(message)

# await Console(
#     team.run_stream(
#         task="What is the step by step guide to invest in share market in India?",
#     )
# )

# await model_client.close()

initial_task = "Explore the unexpected connections between urban planning and human psychology. What patterns exist that we haven't fully recognized yet?"

# Run the research exploration
stream = team.run_stream(task=initial_task)

message_count = 0
async for message in stream:
    message_count += 1
    print(f"💬 Message {message_count}")
    print(f"🤖 Agent: {message.source}")
    display(Markdown(f"📝 **Content:**\n\n{message.content}"))
    print("-" * 40)
    print()

💬 Message 1

🤖 Agent: user

📝 **Content:**

Explore the unexpected connections between urban planning and human psychology. What patterns exist that we haven't fully recognized yet?

----------------------------------------

💬 Message 2

...

🤖 Agent: CuriousExplorer

📝 **Content:**

{'query': 'biometric data architectural monotony Medellín cortisol levels', 'results': [{'id': '1304026065', 'source': 's2orc', 'text': 'and have been performed in all follow-up visits of the original and added cohorts. Other additions to the protocol included a screening for psychotic symptoms in one cohort (RS-III) and, from January 2012 to October 2014, ambulatory polysomnography. In a subsample, taedium vitae was assessed. The most recently introduced assessments include sexual activity, aggression and neuroticism. Major determinants Psychiatric research in the Rotterdam Study focuses on biological risk factors. The vascular depression hypothesis was tested with different measures of atherosclerosis, arterial stiffness and cerebral blood flow [323]. We examined whether blood levels of vitamins and fatty acids, immune parameters, and markers of folate metabolism increased the likelihood of depression [324]. Diurnal patterns of cortisol secretion were studied and recently we performed a low-dose dexamethasone test to assess the negative feedback of the hypothalamic–pituitary–adrenal (HPA) axis functioning [325]. Moreover, several GWAs were conducted in collaborative efforts focusing on depressive symptoms, sleep, anxiety and cortisol [326–328]. Several, mostly cross-sectional studies of brain morphology as possible determinants and correlates of common psychiatric disorders were completed [329]. Also, psychiatric problems and psychological traits such as happiness, sleep duration, and depression are increasingly investigated as determinants of health and mortality [330, 331]. Major clinical outcomes Information on depression is obtained from (a) psychiatric examinations, (b) self-reported histories of depression, (c) medical records, and (d) registration of antidepressant use [332]. The psychiatric examination during each visit consists of an assessment and screening with the Center for Epidemiologic Studies Depression Scale (CES-D), and in the screen-positive participants a semi-structured interview performed by a', 'retrieval_score': 1.4357904195785522, 'exact_score': None}, {'id': '1362663407', 'source': 's2orc', 'text': '(2) these subjects show altered amygdala functional connectivity | | Grossi | 2005 | Repeated measures, case control | To compare salivary cortisol awakening response between individuals with low, moderate and high burnout scores | | Heiden | 2005 | Cross-sectional, case control | To characterize patients with stress-related illnesses by comparing autonomic activity, pressure-pain thresholds, and subjective assessments of health and behavior between patients with stress-related illnesses and control subjects. | | Jovanovic | 2011 | Repeated measures, case control | To investigate whether enduring daily stress causes widespread limbic dysfunctions, and specific changes of the 5-HT1A receptor. | | Sandström | 2011 | Cross-sectional, case control | To use a multivariate statistical approach to examine whether patients with work-related exhaustion and controls differed on an extensive set of biological, psychological and immunological variables. | | Sandström | 2012 | Cross-sectional, case control | To compare functional magnetic resonance imaging (fMRI) patterns and diurnal cortisol across three groups: (i) controls, (ii) acute un-medicated patients with unipolar major depression, and (iii) patients on long-term sick leave due to work stress. | | Savic | 2015 | Repeated measures, case control | To compare salivary cortisol, cortical thickness, cortical surface area and subcortical volumes between individuals with occupational stress compared to controls. | ### Symptom measurement scales, n = 4 | Author | Year | Design | Description | |--------|------|--------|-------------| | Glise24 | 2010 | Longitudinal, cohort | To assess the construct and predictive validity of a new instrument for self-rating of stress-related Exhaustion Disorder (s-ED). |', 'retrieval_score': 1.4355978965759277, 'exact_score': None}, {'id': '1297396645', 'source': 's2orc', 'text': "especially multiday wear ECG patches as used in the current study, are likely to be most valuable in tracking daily and day-to-day fluctuations in HRV (50). Additionally, while salivary cortisol is frequently used as a biomarker of stress (51, 52), in the present sample, we did not find any significant changes in cortisol levels at the cohort level besides the known diurnal variations. This may further suggest the utility of high-frequency sensor-based objective assessment of physiological changes such as HRV over infrequent biological assays that may not be able to capture the dynamic response. Individualized differences in cortisol levels were not assessed due to limited data (one measurement per time of day) across sessions. Most of what is currently known about HRV as a measure of risk in cardiovascular disease is based on population-level cutoffs of risk (23). Our findings suggest that with such high heterogeneity in HRV levels among individuals, a more meaningful measure of changes in autonomic activity would be based on deviations from individual-level baselines over time. In addition, external factors such as medications (53) and mood disorders (54) can impact an individual's HRV, making it even more important to track individualized responses. Heart rate recovery after exercise is another measure of autonomic nervous system input to the heart that is determined by recording the velocity of decrease in heart rate following exercise and has also been found to be associated with cardiovascular outcomes (55). Although it is highly personalized based on an individual's exercise capacity, the expected decrease in heart rate at", 'retrieval_score': 1.4298031330108643, 'exact_score': None}, {'id': '1252849528', 'source': 's2orc', 'text': 'reflect baseline locomotor activity. LA also moved faster on both open (t98 = 4.11, P < 0.001, r = 0.38) and closed (t98 = 3.20, P = 0.002, r = 0.31) arms. Table 1 displays a detailed description of behavioral data of LA and HA. Results of questionnaires for Study 1 can be found in Additional file 3. Endocrinological and physiological responses to the EPM (Study 1) Earlier studies have shown that the EPM activates the rodent stress system and increases corticosterone levels [26]. We hypothesized that EPM testing also elicits a transient activation of the two endogenous human stress systems, namely the sympathetic nervous system and the hypothalamus–pituitary–adrenal axis. As expected, we found a main effect of time for salivary cortisol (F5,22,60.6 = 4.451, P = 0.013, η2 partial = 0.142) and alpha-amylase (F5,130 = 7.253, P < 0.001, η2 partial = 0.218; Fig. 3). In line with earlier experiments examining stress responses [27], the peak concentration of alpha-amylase was enhanced directly after EPM testing (significantly increased directly after compared to at all other time points; all P < 0.01), while cortisol peaked 15 min later (significantly higher at 15 min compared to directly after and at 30, 45, and 60 min (all P < 0.05), whereas before testing it was lower than at 15 min but on trend level (P = 0.09; Fig. 3). Other markers for the immediate activation of the sympathetic nervous system are increased heart and respiration rate, as well as skin conductance levels (SCL). As expected, we found a significant', 'retrieval_score': 1.426957130432129, 'exact_score': None}, {'id': '1239003936', 'source': 's2orc', 'text': 'according to time of day, workers, and workplaces. Results After controlling for awakening time, sex, age, season of sampling, alcohol use, physical activity, psychotropic drugs use, health conditions, and body mass index, results revealed significant associations between psychological distress and depressive and burnout symptoms in relation to cortisol concentrations at different diurnal time points. Specifically, higher levels of psychological distress and depressive symptoms were associated with higher cortisol concentrations at awakening but not 30 minutes afterwards. Starting steadily from 14:00 hours until bedtime, levels of psychological distress and depressive and burnout symptoms were associated with lower cortisol concentrations. Significant cortisol variations were also found between workplaces. Sex did not moderate associations in secondary analyses. Conclusions Distinct time points of diurnal cortisol variation were consistently, albeit modestly, associated with increased psychological distress and depressive and burnout symptoms. Our findings therefore link subjective psychometrics with objective biometrics. This research area could potentially lead to the development of refined screening tools that provide correspondence between diurnal cortisol and mental health. Key terms HPA-axis; mental health; mental illness; salivary cortisol; stress; workplace stress. Mental health problems in the workforce represent a significant burden for the affected worker and its family, and for both businesses and the whole of society, given the costs associated with mental illness (absenteeism, lower productivity, indemnity payments, and health care) (1). Evaluation of workers’ mental health is, however, a difficult task to achieve and may be riddled with controversy given competing interests between employers and employees. In attempting to develop screening tools that can detect impending', 'retrieval_score': 1.4244670867919922, 'exact_score': None}, {'id': '1247088094', 'source': 's2orc', 'text': 'Fernandez FJ, Trianes M, et al. Cognitive performance and morning levels of salivary cortisol and α-amylase in children reporting high vs. low daily stress perception. Span J Psychol 2008;11:1–15. [32] Binboga E, Guven S, Çatikkas F, et al. Psychophysiological responses to competition and the big five personality traits. J Hum Kinet 2012;33:187–94. [33] Casolino E, Lupò C, Cortis C, et al. Technical and tactical analysis of youth taekwondo performance. J Strength Cond Res 2012;26:1489–95. [34] Nater U, Rohleder N. Salivary alpha-amylase as a non-invasive biomarker for the sympathetic nervous system: current state of research. Psychoneuroendocrinology 2009;34:486–96. [35] de Dios Benitez-Sillero J, Perez-Navero JL, Tasset I, et al. Influence of intense exercise on saliva glutathione in prepubescent and pubescent boys. Eur J Appl Physiol 2009;106:181–6. [36] Gallina S, Di Mauro M, D’Amico MA, et al. Salivary chromogranin A, but not α-amylase, correlates with cardiovascular parameters during high-intensity exercise. Clin Endocrinol 2011;75:747–52. [37] Kvilighan KT, Granger DA. Salivary α-amylase response to competition: relation to gender, previous experience, and attitudes. Psychoneuroendocrinology 2006;31:703–14. [38] Wilmore JH, Costill D. Physiology of Sport and Exercise. Champaign, IL: Human Kinetics; 1994. [39] Foster JP, Carl H, Kara M, et al. Differences in perceptions of training by coaches and athletes. Eur J Appl Physiol Occup Physiol 2001;83:7–9.', 'retrieval_score': 1.418941617012024, 'exact_score': None}, {'id': '1277960849', 'source': 's2orc', 'text': '1997. Small Sample Inference for Fixed Effects from Restricted Maximum Likelihood. Biometrics 53:983. DOI: 10.2307/2533558. Klosterman LL., Murai JT., Siiteri PK. 1986. Cortisol levels, binding, and properties of corticosteroid-binding globulin in the serum of primates. Endocrinology 118:424–434. DOI: 10.1210/endo-118-1-424. Kuznetsova A., Brockhoff PB., Christensen RHB. 2016. lmerTest: Tests in Linear Mixed Effects Models, R package version 2.0-33. https://CRAN.R-project.org/package=lmerTest Lattin CR., Romero LM. 2014. Chronic stress alters concentrations of corticosterone receptors in a tissue-specific manner in wild house sparrows (Passer domesticus). Journal of Experimental Biology 217:2601–2608. DOI: 10.1242/jeb.103788. Liebl AL., Shimizu T., Martin LB. 2013. Covariation among glucocorticoid regulatory elements varies seasonally in house sparrows. General and Comparative Endocrinology 183:32–37. DOI: 10.1016/j.ygcen.2012.11.021. McEwen BS., Wingfield JC. 2007. Allostasis and allostatic load. In: Fink G, ed. Encyclopedia of Stress 2. New York: Academic Press, 135-141. Nakagawa S., Santos ESA. 2012. Methodological issues and advances in biological meta-analysis. Evolutionary Ecology 26:1253–1274. DOI: 10.1007/s10682-012-9555-5. Nakagawa S., Schielzeth H. 2010. Repeatability for Gaussian and non-Gaussian data: a practical guide for biologists. Biological Reviews 85:935–956. DOI: 10.1111/j.1469-185X.2010.00141.x. Narayan EJ., Cockrem JF., Hero JM. 2013a. Are baseline and short-term corticosterone stress responses in free-living amphibians repeatable? Comparative Biochemistry and Physiology, Part A 164:21–28. DOI: 10.1016/j.cbpa.2012.10.001. Narayan EJ., Cockrem JF., Hero JM. 2013b. Repeatability of baseline corticosterone and short-term corticosterone stress responses, and their correlation with testosterone and body condition in a terrestrial breeding anuran (Platymantis vitiana). Comparative Biochemistry and Physiology - A Molecular and Integrative Physiology 165:304–312. DOI: 10.1016/j.cbpa.2013.03.033. Nespolo RF., Franco M. 2007. Whole-animal metabolic rate is a repeatable trait: a meta-analysis.', 'retrieval_score': 1.4186731576919556, 'exact_score': None}, {'id': '971969341', 'source': 'massiveds-pubmed', 'text': 'Previous research on hypothalamic-pituitary adrenal (HPA) axis-activity in suicide attempter research has shown conflicting outcomes. The design of the present study was to test the influence of personality disorders and concominant axis I diagnoses on the dexamethasone suppression diagnostic test by use of multiple regression analyses. The sample consisted of 184 patients with a recent suicide attempt and 42 healthy controls. As expected, the lowest pre- and postdexamethasone cortisol levels were found in patients with personality disorders axis II, cluster B as compared to the other patients. The results remained significant when analysed for covariance with DSM-III-R axis I diagnoses, age or sex. Whether these low cortisol levels are due to previous experience of extreme stressful events or long-lasting burden, or whether they may be a consequence of biogenetic or psychological predisposal of interest, remains to be elucidated. Axis I comorbidity needs to be further examined.', 'retrieval_score': 1.40897798538208, 'exact_score': None}], 'total_results': 8}

----------------------------------------

💬 Message 36

AttributeError: 'TaskResult' object has no attribute 'source'